# CV Lab project - Image Integration

Data Exploration

Description of the data:

* datasets:
    * train set, in each folder contains f-c.png and the json of the homography...
    * validation set, contains f-c.png, homographies and true label
    * test set, for submission

* 7 frames for each of the 10 cameras à 70 images for each sample 
    * f $\in$ {0,1,2,3,4,5,6}, where 3 is the center frame and
    * c $\in$ B01,...,B05,G01,...,G05 ,where B01 is the center camera

# ref:
* https://www.py4u.net/discuss/1950670

In [2]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import re

### Merging and Saving the Data

To merge and save merged the images in respective folders

In [3]:
def load_images_from_folder(folder, filter=None, masking=False):
    images = []
    mask = cv2.imread('data/mask.png',0)
    for filename in os.listdir(folder):
        img = None
        if filter is None:
            #print('?')
            img = cv2.imread(os.path.join(folder,filename))
            #apply masking
            if masking:
                img = cv2.bitwise_and(img,img,mask=mask)
        else:
            if filename in filter:
                #print(filename)
                img = cv2.imread(os.path.join(folder,filename))
                if masking:
                    img = cv2.bitwise_and(img,img,mask=mask)
                
        if img is not None:
            images.append(img)
    #print('Loaded {} images'.format(len(images)))
    return images

def makenumpy(df):
    m = None
    for r in df:
        if m is None:
            m = np.array(r).reshape(1,-1)
        else:
            m = np.concatenate((m,np.array(r).reshape(1,-1)),axis=0)
    return m

In [4]:
def merg_v1(warped):
    c = (warped > 0).sum(axis=0)
    warped_ = warped.sum(axis=0).squeeze()
    c[c==0] = 1.0
    c = 1.0/c
    return c * warped_

def merg_v2(warped):
    arr = np.zeros((warped[0].shape[0], warped[0].shape[1], 3))
    divide_arr = np.ones((warped[0].shape[0], warped[0].shape[1]))

    for l in range(len(warped)):
        (x, y, c) = (np.where(warped[l] != 0.0))
        divide_arr[x, y] += 1
        arr += warped[l]

    (x, y, c) = (np.where(arr == 0.0))
    divide_arr[x, y] = 1
    arr[:,:,0] /= divide_arr
    arr[:,:,1] /= divide_arr
    arr[:,:,2] /= divide_arr

    return arr

def merg_avg(warped):
    return np.array(warped).mean(axis=0)
        


In [11]:
dirs = ['train','validation']
fns = ['train','valid']
time_frames = 7
mask = cv2.imread('data/mask.png',0)

for d, dir in enumerate(dirs):
    train_folder_pattern = re.compile(fns[d]+'-(.*)-(.*)')
    for folder in os.listdir('./data/'+dir):  
        if train_folder_pattern.match(folder) is not None:
            print('prc Folder:',folder, end='')         
            homo = pd.read_json('./data/'+dir+'/'+folder+'/homographies.json', orient='record')
            
            for tf in range(time_frames):
                warped = []
                for fn in os.listdir('./data/'+dir+'/'+folder):
                    if ('merg' not in fn) and ('.png' in fn) and ( (str(tf)+'-') in fn):
                        print(' ', fn, end='')
                        img_f = './data/'+dir+'/'+folder+'/'+fn
                        img = cv2.imread(img_f)

                        img = cv2.bitwise_and(img,img,mask=mask)
                        h = makenumpy( homo[ fn.replace('.png', '')] )
                        
                        warped += [cv2.warpPerspective(img/255., h, img.shape[:2] )]

                merged_1 = merg_v1(np.array(warped))
                merged_2 = merg_v2(warped)  
                merged_avg = merg_avg(warped)

                plt.imsave('./data/'+dir+'/'+folder+'/'+str(tf)+'-merg_avg_masked.png',merged_avg)
                plt.imsave('./data/'+dir+'/'+folder+'/'+str(tf)+'-merg_v1_masked.png',merged_1)
                plt.imsave('./data/'+dir+'/'+folder+'/'+str(tf)+'-merg_v2_masked.png',merged_2) 

                    
            print('\t')
            

prc Folder: train-1-5  0-G01.png  0-G03.png  0-G02.png  0-G05.png  0-G04.png  0-B04.png  0-B05.png  0-B01.png  0-B02.png  0-B03.png  1-G05.png  1-G04.png  1-G03.png  1-G02.png  1-G01.png  1-B02.png  1-B03.png  1-B01.png  1-B04.png  1-B05.png  2-B01.png  2-B02.png  2-B03.png  2-B04.png  2-B05.png  2-G05.png  2-G04.png  2-G01.png  2-G03.png  2-G02.png  3-B04.png  3-B05.png  3-B02.png  3-B03.png  3-B01.png  3-G03.png  3-G02.png  3-G01.png  3-G05.png  3-G04.png  4-B05.png  4-B04.png  4-B01.png  4-B03.png  4-B02.png  4-G01.png  4-G02.png  4-G03.png  4-G04.png  4-G05.png  5-B03.png  5-B02.png  5-B01.png  5-B05.png  5-B04.png  5-G04.png  5-G05.png  5-G02.png  5-G03.png  5-G01.png  6-G04.png  6-G05.png  6-G01.png  6-G02.png  6-G03.png  6-B01.png  6-B03.png  6-B02.png  6-B05.png  6-B04.png	
prc Folder: train-1-2  0-G01.png  0-G03.png  0-G02.png  0-G05.png  0-G04.png  0-B04.png  0-B05.png  0-B01.png  0-B02.png  0-B03.png  1-G05.png  1-G04.png  1-G03.png  1-G02.png  1-G01.png  1-B02.png  1-B03.pn